**<font size="4">Выполнил студент группы М3О-210Б-21 - Караваев Кирилл Романович.</font>**

<font size="4">Для распределения Рэлея ($\sigma = 2$) оценить близость гипотетического ряда распределения и эмпирического аналога для объёмов выборки $25, 100, 500$ по критерию $\chi^2$ – Пирсона. </font>

---
Подключаем необходимые библиотеки.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chisquare, rayleigh, chi2

---
Задаем параметры распределения и уровень значимости.

In [2]:
# Параметры распределения Рэлея
sigma = 2

# Объемы выборки
sample_sizes = [25, 100, 500]

# Количество интервалов (как правило, используется sqrt(n))
num_intervals = [int(np.sqrt(n)) for n in sample_sizes]

# Уровень значимости
alpha = 0.05


---
Нас интересует проверка гипотезы о виде распределения наблюдаемых случайных величин. Нам доступна выборка $X = (X_1,\ldots,X_n)$, состоящая из н.о.р.с.в. с функцией распределения $F_{X}(x)$. Мы выдвигаем простую гипотезу 

$$
H_0: F_{X}(x) = F_0(x)
$$

Альтернативные распределения не конкретизируются, то есть в качестве множества $H_1$ выступают все прочие функции распределения.

Используем критерий согласия $\chi^2$ – Пирсона для проверки гипотезы о соответствии выборки распределению Рэлея. 
Имеем:  

$\cdot$выборка: $X = (X_1,\ldots,X_n)$ 
        
$\cdot$нулевая гипотеза: $H_0: X \sim Rayleigh(\sigma)$;
        
$\cdot$альтернативная гипотеза: $H_1: H_0$ - неверна;

$\cdot$статистика:
$$
\chi^2(X) = \sum\limits_{i=1}^{K} \cfrac{(n_i-np_i)^2}{np_i}
$$


$\chi^2(X) \sim \chi^2_{K-1}(X)$ - параметр $\sigma$ задан при $H_0$; 

$[a_i, a_{i+1}], \quad i = \overline{1, K}$ - интервалы;

$n_i$ - число элементов выборки в $[a_i, a_{i+1}]$;

$p_i = F(a_{i+1}) - F(a_i)$ - вероятность попадания в $i$-й интервал при $H_0$.

Проверяемая гипотеза $H_{0}$ отклоняется при больших значениях статистики, когда вычисленное по выборке значение статистики $\chi^2$ больше критического значения $\chi^2_{кр}$ с квантилем распределения уровня $1 - \alpha$ и степенью свободы $K-1$, или достигнутый уровень значимости ($p-value$) меньше заданного уровня значимости (заданной вероятности ошибки 1-го рода).


In [3]:
# Цикл по объемам выборки
for n, k in zip(sample_sizes, num_intervals):
    results = []  # Список для хранения результатов
    
    # Генерация выборки
    sample = rayleigh.rvs(scale=sigma, size=n)
    
    # Генерация теоретических частот
    intervals = np.linspace(min(sample), max(sample), k+1)
    expected_frequencies = []
    for i in range(len(intervals)-1):
        expected_frequencies.append(
            (rayleigh.cdf(intervals[i+1], scale=sigma) - rayleigh.cdf(intervals[i], scale=sigma)) * n
        )
    
    # Нормировка ожидаемых частот
    expected_frequencies = np.array(expected_frequencies)
    expected_frequencies *= n / expected_frequencies.sum()
    
    # Генерация наблюдаемых частот
    observed_frequencies, _ = np.histogram(sample, bins=intervals)
    
    # Вычисление χ² статистики
    chi2_stat, p_value = chisquare(f_obs=observed_frequencies, f_exp=expected_frequencies)
    
    # Получение критического значения χ²
    chi2_critical = chi2.ppf(1 - alpha, df=k-1)
    
    print(f"Объем выборки: {n}")
    
    # Вывод по гипотезе
    if p_value < alpha:
        print(f"p-value: {p_value} < {alpha}")
        print(f"χ² статистика: {chi2_stat}")
        print(f"Критическое значение χ²: {chi2_critical}")
        print("Значение χ² > Критическое значение χ²")
        print("Отвергаем нулевую гипотезу: распределение выборки существенно отличается от распределения Рэлея.")
        
        #if np.sum(expected_frequencies) - np.sum(observed_frequencies) > 0:
         #   print("Ошибка первого рода: выборка была отнесена к распределению Рэлея, но на самом деле не соответствует.")
        #else:
          #  print("Ошибка второго рода: выборка не была отнесена к распределению Рэлея, хотя соответствует.")
    else:
        print(f"p-value: {p_value} > {alpha}")
        print(f"χ² статистика: {chi2_stat}")
        print(f"Критическое значение χ²: {chi2_critical}")
        print("Значение χ² < Критическое значение χ²")
        print("Нет достаточных оснований для отвержения нулевой гипотезы. Гипотеза принимается.")
    print("------------------------------------------------------------------------------------\n")
    
    # Создание строк результатов для каждого интервала
    for i in range(len(intervals)-1):
        result_row = {
            'Нижняя граница интервала': intervals[i],
            'Верхняя граница интервала': intervals[i+1],
            'Количество попаданий в интервал': observed_frequencies[i],
            'Частота': observed_frequencies[i] / n,
            'Ожидаемая частота': expected_frequencies[i] / n
        }
        results.append(result_row)

    # Создание датафрейма из списка результатов
    df = pd.DataFrame(results)

    # Сохранение в CSV файл "dataframe_{n}.csv"
    filename = f'dataframe_{n}.csv'
    df.to_csv(filename, index=False)


    # Создание датафрейма для второго CSV файла
    comparison_row = {
        'Статистика χ²': chi2_stat,
        'Критическое χ²': chi2_critical,
        'p-value': p_value,
        'Принимается гипотеза?': chi2_stat < chi2_critical 
    }
    comparison_df = pd.DataFrame([comparison_row])

    # Сохранение во второй CSV файл
    comparison_filename = f'comparison_{n}.csv'
    comparison_df.to_csv(comparison_filename, index=False)

Объем выборки: 25
p-value: 0.9765427520727525 > 0.05
χ² статистика: 0.4679754690617178
Критическое значение χ²: 9.487729036781154
Значение χ² < Критическое значение χ²
Нет достаточных оснований для отвержения нулевой гипотезы. Гипотеза принимается.
------------------------------------------------------------------------------------

Объем выборки: 100
p-value: 0.3195740235675789 > 0.05
χ² статистика: 10.393552029557268
Критическое значение χ²: 16.918977604620448
Значение χ² < Критическое значение χ²
Нет достаточных оснований для отвержения нулевой гипотезы. Гипотеза принимается.
------------------------------------------------------------------------------------

Объем выборки: 500
p-value: 0.20278055526618427 > 0.05
χ² статистика: 26.096840626461493
Критическое значение χ²: 32.670573340917315
Значение χ² < Критическое значение χ²
Нет достаточных оснований для отвержения нулевой гипотезы. Гипотеза принимается.
----------------------------------------------------------------------------

In [4]:
df1 = pd.read_csv('dataframe_25.csv')
df2 = pd.read_csv('dataframe_100.csv')
df3 = pd.read_csv('dataframe_500.csv')

res1 = pd.read_csv('comparison_25.csv')
res2 = pd.read_csv('comparison_100.csv')
res3 = pd.read_csv('comparison_500.csv')

---
**<font size="3">Выборка: 25</font>**

In [5]:
df1

,Нижняя граница интервала,Верхняя граница интервала,Количество попаданий в интервал,Частота,Ожидаемая частота
0,0.494656,1.665964,6,0.24,0.273010
1,1.665964,2.837273,10,0.40,0.354232
2,2.837273,4.008582,6,0.24,0.240191
3,4.008582,5.179890,2,0.08,0.102998
4,5.179890,6.351199,1,0.04,0.029570


In [6]:
res1

,Статистика χ²,Критическое χ²,p-value,Принимается гипотеза?
0,0.467975,9.487729,0.976543,True


---
**<font size="3">Выборка: 100</font>**

In [7]:
df2

,Нижняя граница интервала,Верхняя граница интервала,Количество попаданий в интервал,Частота,Ожидаемая частота
0,0.313244,0.922831,13,0.13,0.090426
1,0.922831,1.532418,17,0.17,0.156222
2,1.532418,2.142005,11,0.11,0.185437
3,2.142005,2.751592,24,0.24,0.178633
4,2.751592,3.361179,12,0.12,0.147185
5,3.361179,3.970766,8,0.08,0.106194
6,3.970766,4.580353,10,0.10,0.067939
7,4.580353,5.189940,3,0.03,0.038831
8,5.189940,5.799527,1,0.01,0.019924
9,5.799527,6.409114,1,0.01,0.009208


In [8]:
res2

,Статистика χ²,Критическое χ²,p-value,Принимается гипотеза?
0,10.393552,16.918978,0.319574,True


---
**<font size="3">Выборка: 500</font>**

In [9]:
df3

,Нижняя граница интервала,Верхняя граница интервала,Количество попаданий в интервал,Частота,Ожидаемая частота
0,0.111470,0.456911,18,0.036,0.024258
1,0.456911,0.802353,30,0.060,0.051671
2,0.802353,1.147794,38,0.076,0.074676
3,1.147794,1.493236,46,0.092,0.091610
4,1.493236,1.838678,50,0.100,0.101624
5,1.838678,2.184119,66,0.132,0.104723
6,2.184119,2.529561,50,0.100,0.101662
7,2.529561,2.875003,35,0.070,0.093740
8,2.875003,3.220444,51,0.102,0.082529
9,3.220444,3.565886,29,0.058,0.069623


In [10]:
res3

,Статистика χ²,Критическое χ²,p-value,Принимается гипотеза?
0,26.096841,32.670573,0.202781,True


---
**<font size="4">Вывод:</font>**

*Критерий хи-квадрат Пирсона (χ²-тест)* является статистическим методом, используемым для проверки независимости или согласия между наблюдаемыми и ожидаемыми значениями в категориальных переменных или для проверки соответствия распределения наблюдаемых данных ожидаемому теоретическому распределению. 

Критерий оценивает соответствие между эмпирическим распределением выборки и теоретическим распределением, в нашем случае -  Рэлея, основываясь на сравнении функций распределения или частот. 

Если говорить про объем выборки, то, в целом, можно сказать, что он может влиять на исход и результаты статистического теста. Чем больше объем выборки, тем больше информации о распределении мы получаем, и тем более точные могут быть наши выводы. Больший объем выборки обычно позволяет лучше оценить параметры распределения и уменьшить стандартные ошибки. Это может привести к более точным и более надежным результатам статистического теста. Собственно и стоит сказать, что критерий хи-квадрат может быть чувствителен к небольшому объему выборки. При маленьких объемах выборки статистика χ² может давать недостоверные результаты или не иметь достаточной мощности для обнаружения реальных различий. Есть и другие факторы, которые могут влиять на искажение результатов теста, но в данной работе они отсутствуют.